In [1]:
import re
import numpy as np
import heapq
from functools import reduce,cache
from operator import mul
from bisect import insort
from itertools import permutations
from collections import deque,defaultdict
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
029A
980A
179A
456A
379A
"""

### Part 1
Oh this is silly. I'm at a loss slightly but thoroughly entertained. My cat is destroying a box loudly in the background. Life is good.


In [114]:
class Keypads:
    def __init__(self,txt):
        self.codes = txt.splitlines()
        # self.num_pad =[str(x) for x in [[7,8,9],[4,5,6],[1,2,3],[None,0,'A']]]
        nums = [[7,8,9],[4,5,6],[1,2,3],[None,0,'A']]
        self.num_dict = {str(val):(x,y) for y,line in enumerate(reversed(nums)) for x,val in enumerate(line) if val != None}
        dirs = [[None,'^','A'],['<','v','>']]
        self.dir_dict = {str(val):(x,y) for y,line in enumerate(reversed(dirs)) for x,val in enumerate(line) if val != None}

        print(self.num_dict)
        print(self.dir_dict)
        complexity = 0
        for code in self.codes:
            p1 = self.decode_num(code,(2,0))
            print(f"{code}: {p1}")
            p2 = self.decode_dir(p1,(2,1))
            print(f"      {p2}")
            p3 = self.decode_dir(p2,(2,1))
            print(f"      {p3}")
            print(f"len:{len(p3)}")
            complexity += len(p3) * int(code[:-1])
        print(complexity)

    def decode_dir(self,code,cursor): # from num to dirs
        # >^ for going up, v<
        res = []
        for c in code:
            dir = self.return_path(cursor,self.dir_dict[c])
            cursor = self.dir_dict[c]
            res.append(dir)
        return self.to_str(res)


    def decode_num(self,code,cursor): # from num to dirs
        # >^ for going up, v<
        res = []
        for c in code:
            dir = self.return_path(cursor,self.num_dict[c])
            cursor = self.num_dict[c]
            res.append(dir)
        return self.to_str(res,True)
    
    def to_str(self,directions,nums=False): # from dirs to more dirs
        res = []
        for x,y in directions: #[(1,0)...] where (1,0) represent one action + A
            sub = []
            for _ in range(abs(x)):
                if x>0:
                    sub.append(">")
                else:
                    sub.append("<")
            for _ in range(abs(y)):
                if y>0:
                    sub.append("^")
                else:
                    sub.append("v")
            if nums and y > 0:
                res += reversed(sub)
            else:
                if y<0: # if dirs
                    res += reversed(sub)
                else:
                    res += sub
            res.append("A")
        return ''.join(res)

    
    def return_path(self,src,dest):
        return (dest[0]-src[0],dest[1]-src[1])


test = Keypads(test_input)

{'0': (1, 0), 'A': (2, 0), '1': (0, 1), '2': (1, 1), '3': (2, 1), '4': (0, 2), '5': (1, 2), '6': (2, 2), '7': (0, 3), '8': (1, 3), '9': (2, 3)}
{'<': (0, 0), 'v': (1, 0), '>': (2, 0), '^': (1, 1), 'A': (2, 1)}
029A: <A^A^^>AvvvA
      v<<A>>^A<A>A<AAv>A^Av<AAA>^A
      v<A<AA>>^AvAA<^A>Av<<A>>^AvA^Av<<A>>^AAv<A>A^A<A>Av<A<A>>^AAAvA<^A>A
len:68
980A: ^^^A<AvvvA>A
      <AAA>Av<<A>>^Av<AAA>^AvA^A
      v<<A>>^AAAvA^Av<A<AA>>^AvAA<^A>Av<A<A>>^AAAvA<^A>Av<A>^A<A>A
len:60
179A: ^<<A^^A>>AvvvA
      <Av<AA>>^A<AA>AvAA^Av<AAA>^A
      v<<A>>^Av<A<A>>^AAvAA<^A>Av<<A>>^AAvA^Av<A>^AA<A>Av<A<A>>^AAAvA<^A>A
len:68
456A: ^^<<A>A>AvvA
      <AAv<AA>>^AvA^AvA^Av<AA>^A
      v<<A>>^AAv<A<A>>^AAvAA<^A>Av<A>^A<A>Av<A>^A<A>Av<A<A>>^AAvA<^A>A
len:64
379A: ^A^^<<A>>AvvvA
      <A>A<AAv<AA>>^AvAA^Av<AAA>^A
      v<<A>>^AvA^Av<<A>>^AAv<A<A>>^AAvAA<^A>Av<A>^AA<A>Av<A<A>>^AAAvA<^A>A
len:68
127900


At this point I realized it's not at all enough to just have A path and I can't optimize a path by hardcoding it in. I'm now thinking about recursion, after conversion to dirs for the first time, recursively decode a subsection and keep the shortest combination

In [150]:
class Keypads:
    def __init__(self,txt):
        self.codes = txt.splitlines()
        # self.num_pad =[str(x) for x in [[7,8,9],[4,5,6],[1,2,3],[None,0,'A']]]
        nums = [[7,8,9],[4,5,6],[1,2,3],[None,0,'A']]
        self.num_dict = {
            str(val) if val is not None else val:(x,y)
            for y,line in enumerate(reversed(nums)) 
            for x,val in enumerate(line)
            }
        dirs = [[None,'^','A'],['<','v','>']]
        self.dir_dict = {str(val) if val is not None else val: (x,y)
                         for y,line in enumerate(reversed(dirs))
                         for x,val in enumerate(line)}
        self.directions = {'>':(1,0),'<':(-1,0),'^':(0,1),'v':(0,-1)}

        print(self.num_dict)
        print(self.dir_dict)
        complexity = 0
        for code in self.codes:
            p1 = self.decode_num(code,(2,0),4)
            # print(f"{code}: {p1}")
            # p2 = self.decode_dir(p1,(2,1))
            # print(f"      {p2}")
            # p3 = self.decode_dir(p2,(2,1))
            # print(f"      {p3}")
            # print(f"len:{len(p3)}")
            print(f"min_len * code = {len(p1)}*{int(code[:-1])}")
            sum = len(p1) * int(code[:-1])
            print(f"sum={sum}")
            complexity += sum
        print(complexity)

    def decode_dir(self,code,cursor,n): # from num to dirs
        print(f"decode_dir loop:{n} trying to decode:{code}, cursor: {cursor}")
        # >^ for going up, v<
        res = []
        for c in code:
            dir = self.best_path(cursor,self.dir_dict[c],n)
            # print(f"dir: {dir}")
            cursor = self.dir_dict[c]
            res.append(dir)
        # print(f"res: {res}")
        return res

    def decode_num(self,code,cursor,n): # from num to dirs
        # >^ for going up, v<
        res = []
        for c in code:
            print(f"letter: {c} from,to: {cursor,self.num_dict[c]}")
            path = self.best_path(cursor,self.num_dict[c],n,True) # recursively get to the end of the loop
            cursor = self.num_dict[c]
            # print(f"path in decode_num:{path}")
            res += path
        print(f"finalres: {res}")
        return res
    
    # def return_path(self,src,dest):
    #     return (dest[0]-src[0],dest[1]-src[1])

    def best_path(self,src,dest,n,nums = False):
        # figure out the distance between the coords
        x,y = (dest[0]-src[0],dest[1]-src[1])
        
        # test multiple paths, filter out ones that will goes out of range/blank psaces
        xc, yc = '<','v'
        if x > 0: xc = '>'
        # else: xc = '<'
        if y > 0: yc = '^'
        chars = xc * abs(x) + yc * abs(y)
        if n == 0:
            # print(f"src,dest: {src, dest} n: {n} end of loop: {chars+'A'}")
            return chars + 'A'
        possible_paths = set() # getting possible paths and cut ones that go through null
        for path in set(permutations(chars)): #<<^, <^<, ^<<...
            current = src
            for d in path:
                current = (src[0]+self.directions[d][0],src[1]+self.directions[d][1])
                # print(f"current:{current}")
                if nums:
                    if current not in self.num_dict:
                        break
                elif current not in self.dir_dict:
                    break
                if nums and self.num_dict[None] == current:
                    break # CHECK IF THIS IS THE CORRECT KEYWORD?
                elif self.dir_dict[None] == current:
                    break
            possible_paths.add(path)
        

        full_res = []
        print(f"possible paths: {possible_paths}")
        for path in possible_paths:
            path = list(path)
            print(f"path: {path}")
            res = ''.join(self.decode_dir(path+['A'],(2,1),n-1)) 
            # print(f"res:{res}")
            min_len,min_res = float('inf'),[]
            if len(res) < min_len:
                min_len = len(res)
                min_res = res     
        # print(f"full_res : {full_res}")       
        print(f"min_res : {min_res}")       
        full_res += min_res

        return ''.join(full_res) #possible_paths



test = Keypads(test_input)


{None: (0, 0), '0': (1, 0), 'A': (2, 0), '1': (0, 1), '2': (1, 1), '3': (2, 1), '4': (0, 2), '5': (1, 2), '6': (2, 2), '7': (0, 3), '8': (1, 3), '9': (2, 3)}
{'<': (0, 0), 'v': (1, 0), '>': (2, 0), None: (0, 1), '^': (1, 1), 'A': (2, 1)}
letter: 0 from,to: ((2, 0), (1, 0))
possible paths: {('<',)}
path: ['<']
decode_dir loop:3 trying to decode:['<', 'A'], cursor: (2, 1)
possible paths: {('<', 'v', '<'), ('v', '<', '<'), ('<', '<', 'v')}
path: ['<', 'v', '<']
decode_dir loop:2 trying to decode:['<', 'v', '<', 'A'], cursor: (2, 1)
possible paths: {('<', 'v', '<'), ('v', '<', '<'), ('<', '<', 'v')}
path: ['<', 'v', '<']
decode_dir loop:1 trying to decode:['<', 'v', '<', 'A'], cursor: (2, 1)
possible paths: {('<', 'v', '<'), ('v', '<', '<'), ('<', '<', 'v')}
path: ['<', 'v', '<']
decode_dir loop:0 trying to decode:['<', 'v', '<', 'A'], cursor: (2, 1)
path: ['v', '<', '<']
decode_dir loop:0 trying to decode:['v', '<', '<', 'A'], cursor: (2, 1)
path: ['<', '<', 'v']
decode_dir loop:0 trying 

### Part 1 - Call for Help
Ok I've done some research because I got too stuck. Overall now I have a bit of extra insight:
To expand two inputs on a menu, it's the length of the shortest path from here to there. for example:
- to type `0` I type `<A`, where `<` is the shortest distance between `A` to `0`, append `A`.
- to type `<` of `<A`, it is traveling from `A->-v-<` append `A`.

Therefore, I need to write a function that computes the shortest path between a,b with a graph (so it can be used for both grids)

In [ ]:
# ref: https://github.com/michaeljgallagher/Advent-of-Code/blob/master/2024/21.py
class Keypads:
    def __init__(self,txt):
        self.codes = txt.splitlines()
        # self.num_pad =[str(x) for x in [[7,8,9],[4,5,6],[1,2,3],[None,0,'A']]]
        nums = [[None,0,'A'],[1,2,3],[4,5,6],[7,8,9]]
        self.num_dict = {
            str(val):(x,y)
            for y,line in enumerate(reversed(nums)) 
            for x,val in enumerate(line)
            if val is not None 
            }
        dirs = [[None,'^','A'],['<','v','>']]
        self.dir_dict = {str(val):(x,y)
                         for y,line in enumerate(dirs)
                         for x,val in enumerate(line)
                         if val is not None
                         }
        self.directions = {'>':(1,0),'<':(-1,0),'^':(0,-1),'v':(0,1)}

        print(self.num_dict)
        print(self.dir_dict)

    
    def calculate_complexity(self,n):
        complexity = 0
        for code in self.codes:
            shortest = self.decode(code,n,True)
            code_val = int(code[:-1])
            complexity += len(shortest)*code_val
        return complexity
    
    @cache
    def shortest_paths(self,src,dest, nums = False): # bfs
        open = deque()
        open.append((src,[])) # [], shortest paths
        shortest = float('inf')
        seen = {src}
        shortest_paths = []
        while open:
            # print(f"open{open}")
            current, paths = open.popleft()
            seen.add(current)
            # if current is dest, check if path is <= shortest, if so append
            # print(f"curr {current} dest {dest}")
            if current == dest:
                if len(paths) <= shortest:
                    shortest = len(paths)
                    shortest_paths.append(paths+['A'])
                continue
            # if path longer than shortest bot not goal, no need to search
            if len(paths) >= shortest: continue
            # add neighbors to seen
            for dir, delta in self.directions.items():
                # print(f"dir{dir} delta{delta}")
                neighbor = (current[0] + delta[0], current[1] + delta[1])
                # print(f"neighbor: {neighbor}")
                if nums:
                    if neighbor in self.num_dict.values() and neighbor not in seen:
                        open.append((neighbor,paths+[dir])) 
                elif neighbor in self.dir_dict.values() and neighbor not in seen:
                    open.append((neighbor,paths+[dir])) 
            # print(f"open at end of loop: {open}")
        # print(shortest_paths)
        return shortest_paths
    
    @cache # instead of passing the path, only pass the size
    def decode(self,code,n,nums=False): # first initialization pass in True bc it's a numbers loop
        total = []
        code = list(code)
        for a,b in zip(['A']+code,code):
            # print(f"ab: {a, b}")
            # print( f"grid: {grid[str(a)], self.num_dict['0']}")
            if nums:
                paths = self.shortest_paths(self.num_dict[str(a)],self.num_dict[str(b)],nums) 
            else:
                paths = self.shortest_paths(self.dir_dict[str(a)],self.dir_dict[str(b)],nums) 
            # print(a,b, paths)
            if n == 0: # last layer, get the values and get out
                min_path = min(paths,key=lambda path: len(path))
                total += min_path
            else:
                sub_paths = [self.decode(''.join(path),n-1) for path in paths]
                min_size,min_path = float('inf'),[]
                for sp in sub_paths:
                    if len(sp) < min_size:
                        min_size = len(sp)
                        min_path = sp
                # min_path = min(paths,key=lambda path: len(self.decode(path,n-1,self.dir_dict)))
                total += min_path
        return ''.join(total)


test = Keypads(input)
print("part1:", test.calculate_complexity(2))
# print("part2:", test.calculate_complexity(25))
# attempt 1: part 2 stuck in 19 minutes


That solution got me part 1. However it got stuck in a 19 minute loop until it crashed, even caching didn't help.

### Part 2
And then I realized I do Not need the whole string, just keep track of the length of the results is fine.

And it worked within seconds. Alright

In [5]:
class Keypads:
    def __init__(self,txt):
        self.codes = txt.splitlines()
        # self.num_pad =[str(x) for x in [[7,8,9],[4,5,6],[1,2,3],[None,0,'A']]]
        nums = [[None,0,'A'],[1,2,3],[4,5,6],[7,8,9]]
        self.num_dict = {
            str(val):(x,y)
            for y,line in enumerate(reversed(nums)) 
            for x,val in enumerate(line)
            if val is not None 
            }
        dirs = [[None,'^','A'],['<','v','>']]
        self.dir_dict = {str(val):(x,y)
                         for y,line in enumerate(dirs)
                         for x,val in enumerate(line)
                         if val is not None
                         }
        self.directions = {'>':(1,0),'<':(-1,0),'^':(0,-1),'v':(0,1)}

        print(self.num_dict)
        print(self.dir_dict)

    
    def calculate_complexity(self,n):
        complexity = 0
        for code in self.codes:
            shortest = self.decode(code,n,True)
            code_val = int(code[:-1])
            complexity += shortest*code_val#len(shortest)*code_val
        return complexity
            # p1 = self.shortest_paths()
            # print(f"{code}: {p1}")
        #     print(f"min_len * code = {len(p1)}*{int(code[:-1])}")
        #     sum = len(p1) * int(code[:-1])
        #     print(f"sum={sum}")
        #     complexity += sum
        # print(complexity)
    
    @cache
    def shortest_paths(self,src,dest, nums = False): # bfs
        open = deque()
        open.append((src,[])) # [], shortest paths
        shortest = float('inf')
        seen = {src}
        shortest_paths = []
        while open:
            # print(f"open{open}")
            current, paths = open.popleft()
            seen.add(current)
            # if current is dest, check if path is <= shortest, if so append
            # print(f"curr {current} dest {dest}")
            if current == dest:
                if len(paths) <= shortest:
                    shortest = len(paths)
                    shortest_paths.append(paths+['A'])
                continue
            # if path longer than shortest bot not goal, no need to search
            if len(paths) >= shortest: continue
            # add neighbors to seen
            for dir, delta in self.directions.items():
                # print(f"dir{dir} delta{delta}")
                neighbor = (current[0] + delta[0], current[1] + delta[1])
                # print(f"neighbor: {neighbor}")
                if nums:
                    if neighbor in self.num_dict.values() and neighbor not in seen:
                        open.append((neighbor,paths+[dir])) 
                elif neighbor in self.dir_dict.values() and neighbor not in seen:
                    open.append((neighbor,paths+[dir])) 
            # print(f"open at end of loop: {open}")
        # print(shortest_paths)
        return shortest_paths
    
    @cache
    def decode(self,code,n,nums=False): # first initialization pass in True bc it's a numbers loop
        total = 0
        code = list(code)
        for a,b in zip(['A']+code,code):
            # print(f"ab: {a, b}")
            # print( f"grid: {grid[str(a)], self.num_dict['0']}")
            if nums:
                paths = self.shortest_paths(self.num_dict[str(a)],self.num_dict[str(b)],nums) 
            else:
                paths = self.shortest_paths(self.dir_dict[str(a)],self.dir_dict[str(b)],nums) 
            # print(a,b, paths)
            if n == 0: # last layer, get the values and get out
                # min_path = min(paths,key=lambda path: len(path))
                # total += min_path
                total += min(len(path) for path in paths)
            else:
                # sub_paths = [self.decode(''.join(path),n-1) for path in paths]
                # min_size,min_path = float('inf'),[]
                # for sp in sub_paths:
                #     if len(sp) < min_size:
                #         min_size = len(sp)
                #         min_path = sp
                # min_path = min(paths,key=lambda path: len(self.decode(path,n-1,self.dir_dict)))
                total += min([self.decode(''.join(path),n-1) for path in paths])
        return total


test = Keypads(input)
print("part1:", test.calculate_complexity(2))
print("part2:", test.calculate_complexity(25))
# attempt 1: part 2 stuck in 19 minutes


{'7': (0, 0), '8': (1, 0), '9': (2, 0), '4': (0, 1), '5': (1, 1), '6': (2, 1), '1': (0, 2), '2': (1, 2), '3': (2, 2), '0': (1, 3), 'A': (2, 3)}
{'^': (1, 0), 'A': (2, 0), '<': (0, 1), 'v': (1, 1), '>': (2, 1)}
part1: 188384
part2: 232389969568832


### Takeaways
- Honestly, at this time. I should've guessed that BFS is gonna sneak its way into not-so-grid puzzles too.
- So maybe today's takeaway is... Just see if BFS works and kept trying (?)